In [1]:
## magics

%load_ext version_information 
%version_information numpy,xarray,netCDF4

Software versions
Python 2.7.11 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.2.0
OS Linux 3.16.0 4 amd64 x86_64 with debian 8.5
numpy 1.11.1
xarray 0.9.1
netCDF4 1.2.4
Mon Aug 28 14:49:50 2017 CEST

In [2]:
## path for mdules

import sys
sys.path.insert(0,"/home/albert/lib/python")

In [3]:
## imports

import numpy as np
import dask
import xarray as xr
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd 
import datetime
import glob as glob
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
import time
from dask.diagnostics import ProgressBar

%matplotlib inline

In [4]:
## datasets

#NATL60

dir="/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/"
files="NATL60-CJM165_y2012m07d*.1d_gridT.nc"

tfiles=dir+files

!ls $tfiles


/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d01.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d02.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d03.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d04.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d05.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d06.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d07.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d08.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d09.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d10.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d11.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d12.1d_gridT.nc
/media/extra/DAT

In [5]:
dsN = xr.open_mfdataset(tfiles,concat_dim='time_counter',decode_times=False, chunks={'deptht':1 ,'time_counter':1})

In [6]:
latN = dsN.nav_lat
lonN = dsN.nav_lon
tempN=dsN.votemper
saltN=dsN.vosaline
depN = dsN.deptht
timN = dsN.time_counter

In [7]:
observation_lon=-63.784
observation_lat=37.495

In [8]:
indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))  
model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
model_tempBOX=tempN[:,0,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
model_saltBOX=saltN[:,0,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]

model_lonBOX_array=model_lonBOX.values
model_latBOX_array=model_latBOX.values
model_tempBOX_array=model_tempBOX.values

In [32]:
#loop over vertical level
def profile_mean_percent_all_dates(k):
    
    model_mean=np.zeros(11)
    percent_10=np.zeros(11)
    percent_90=np.zeros(11)
    
    for t in np.arange(-10,10):
        
        day=15+t
        if day < 10:
            file="NATL60-CJM165_y2012m07d0"+str(day)+".1d_gridT.nc"
        else:
            file="NATL60-CJM165_y2012m07d"+str(day)+".1d_gridT.nc"
        tfile=dir+file
        !ls $tfile
        
        dsN = xr.open_dataset(tfile, chunks={'deptht':30 ,'time_counter':1})
        
        latN = dsN.nav_lat
        lonN = dsN.nav_lon
        tempN=dsN.votemper
        saltN=dsN.vosaline
        depN = dsN.deptht
        timN = dsN.time_counter

        #decoupage grossier autour de la position du profile ARGO pour un niveau vertical

        indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))  
        model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
        model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
        model_tempBOX=tempN[:,k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
        model_saltBOX=saltN[:,k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]

        model_lonBOX_array=model_lonBOX.values
        model_latBOX_array=model_latBOX.values
        model_tempBOX_array=model_tempBOX.values
        model_saltBOX_array=model_saltBOX.values
        time_array=timN.values
        t_dim=np.arange(1)
        x_dim=np.arange(122)
        y_dim=np.arange(159)

        # construction d'un nouveau xarray
        d = {}
        d['time_counter'] = ('time_counter',t_dim)
        d['y'] = ('y',y_dim)
        d['x'] = ('x',x_dim)
        d['nav_lat'] = (['y','x'],model_latBOX_array)
        d['nav_lon'] = (['y','x'],model_lonBOX_array)
    
        d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
        d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
    
        dset = xr.Dataset(d)
    
        latB = dset.nav_lat
        lonB = dset.nav_lon
        model_temperatureB = dset.votemper
        model_salinityB = dset.vosaline
    
        # selection plus fine des profils
        
        model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
        model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))
        lon_stacked = lonB.stack(profile=('x', 'y'))
        lat_stacked = latB.stack(profile=('x', 'y'))
    
        distance_threshold = 0.25
        square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
        is_close_to_observation = square_distance_to_observation < distance_threshold**2
    
        model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
        model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
        lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
        lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)
    
        model_mean[t] = model_temperature_near_observation.mean()
        percent_10[t]= model_temperature_near_observation.quantile(0.1)
        percent_90[t]= model_temperature_near_observation.quantile(0.9)

    return model_mean,percent_10,percent_90

In [34]:
%timeit model_mean,percent_10,percent_90=profile_mean_percent_all_dates(0)

/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d05.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d06.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d07.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d08.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d09.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d10.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d11.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d12.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d13.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d14.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d15.1d_gridT.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/3d/NATL60-CJM165_y2012m07d16.1d_gridT.nc
/media/extra/DAT

In [13]:
#loop over vertical level
def profile_mean_percent(k):

    #decoupage grossier autour de la position du profile ARGO pour un niveau vertical

    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))  
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_tempBOX=tempN[:,k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_saltBOX=saltN[:,k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]

    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    model_tempBOX_array=model_tempBOX.values
    model_saltBOX_array=model_saltBOX.values
    time_array=timN.values
    t_dim=np.arange(31)
    x_dim=np.arange(122)
    y_dim=np.arange(159)

    # construction d'un nouveau xarray
    d = {}
    d['time_counter'] = ('time_counter',t_dim)
    d['y'] = ('y',y_dim)
    d['x'] = ('x',x_dim)
    d['nav_lat'] = (['y','x'],model_latBOX_array)
    d['nav_lon'] = (['y','x'],model_lonBOX_array)
    
    d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
    d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
    
    dset = xr.Dataset(d)
    
    latB = dset.nav_lat
    lonB = dset.nav_lon
    model_temperatureB = dset.votemper
    model_salinityB = dset.vosaline
    
    # selection plus fine des profils
    
    model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
    model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))
    lon_stacked = lonB.stack(profile=('x', 'y'))
    lat_stacked = latB.stack(profile=('x', 'y'))
    
    distance_threshold = 0.25
    square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
    is_close_to_observation = square_distance_to_observation < distance_threshold**2
    
    model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
    model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
    lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
    lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)
    
    model_mean = model_temperature_near_observation.mean()
    percentile_10= model_temperature_near_observation.quantile(0.1)
    percentile_90= model_temperature_near_observation.quantile(0.9)

    return model_mean,percentile_10,percentile_90

In [18]:
%timeit model_mean,percentile_10,percentile_90=profile_mean_percent(0)

1 loop, best of 3: 30.7 s per loop


In [20]:
model_mean,percentile_10,percentile_90=profile_mean_percent(0)
model_mean

<xarray.DataArray ()>
array(25.199009675342815)

In [22]:
print percentile_10.values,percentile_90.values

24.3521640778 25.9938922882


In [72]:
%timeit profile_mean_percent(0)

1 loop, best of 3: 25.9 s per loop


In [69]:
#loop over vertical level
def profile_mean(k):

    #decoupage grossier autour de la position du profile ARGO pour un niveau vertical

    indxBOX=np.where((lonN>observation_lon-1)&(lonN<observation_lon+1)&(latN>observation_lat-1)&(latN<observation_lat+1))  
    model_lonBOX=lonN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_latBOX=latN[np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_tempBOX=tempN[:,k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]
    model_saltBOX=saltN[:,k,np.min(indxBOX[0]):np.max(indxBOX[0]),np.min(indxBOX[1]):np.max(indxBOX[1])]

    model_lonBOX_array=model_lonBOX.values
    model_latBOX_array=model_latBOX.values
    model_tempBOX_array=model_tempBOX.values
    model_saltBOX_array=model_saltBOX.values
    time_array=timN.values
    t_dim=np.arange(1)
    x_dim=np.arange(122)
    y_dim=np.arange(159)

    # construction d'un nouveau xarray
    d = {}
    d['time_counter'] = ('time_counter',t_dim)
    d['y'] = ('y',y_dim)
    d['x'] = ('x',x_dim)
    d['nav_lat'] = (['y','x'],model_latBOX_array)
    d['nav_lon'] = (['y','x'],model_lonBOX_array)
    
    d['votemper'] = (['time_counter','y','x'], model_tempBOX_array)
    d['vosaline'] = (['time_counter','y','x'], model_saltBOX_array)
    
    dset = xr.Dataset(d)
    
    latB = dset.nav_lat
    lonB = dset.nav_lon
    model_temperatureB = dset.votemper
    model_salinityB = dset.vosaline
    
    # selection plus fine des profils
    
    model_temperature_stacked = model_temperatureB.stack(profile=('x', 'y'))
    model_salinity_stacked = model_salinityB.stack(profile=('x', 'y'))
    lon_stacked = lonB.stack(profile=('x', 'y'))
    lat_stacked = latB.stack(profile=('x', 'y'))
    
    distance_threshold = 0.25
    square_distance_to_observation = (lon_stacked - observation_lon)**2 + (lat_stacked-observation_lat)**2
    is_close_to_observation = square_distance_to_observation < distance_threshold**2
    
    model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
    model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
    lat_near_observation = lat_stacked.where(is_close_to_observation, drop=True)
    lon_near_observation = lon_stacked.where(is_close_to_observation, drop=True)
    
    model_mean = model_temperature_near_observation.mean()

    return model_mean

In [70]:
%timeit profile_mean(0)

1 loop, best of 3: 26.7 s per loop


In [71]:
%timeit profile_mean(0)

1 loop, best of 3: 25.9 s per loop


In [73]:
%timeit profile_mean(1)

1 loop, best of 3: 24.7 s per loop
